# Youtube Analytics API

In [17]:
import os
from venv import create
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

import pandas as pd

In [16]:
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'
CLIENT_SECRETS_FILE = 'client_secret_898467674662-a1b5vidubbadgt1om1cic23mho33ngok.apps.googleusercontent.com.json'


def get_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_local_server()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()
  return response

In [11]:
result = execute_api_request(
        youtubeAnalytics.reports().query,
        ids='channel==MINE',
        startDate='2022-01-01',
        endDate='2023-04-30',
        dimensions='video',
        metrics='estimatedMinutesWatched,views,likes,subscribersGained',
        maxResults=200,
        sort='-estimatedMinutesWatched'
    )

NameError: name 'youtubeAnalytics' is not defined

# Youtube Data API

## Base functions

### Quotas

- search_results: 100 por palavra chave
- channels: 1
- 

### Search e channel stats


In [1]:
def search_result(query):
    # Refer to the documentation: https://googleapis.github.io/google-api-python-client/docs/dyn/youtube_v3.search.html
    request = youtube.search().list(part="snippet", q=query, maxResults=50)
        
    return request.execute()

In [2]:
def channel_stats(channelID, part="statistics"):
    # Refer to the documentation: https://googleapis.github.io/google-api-python-client/docs/dyn/youtube_v3.channels.html
    
    request = youtube.channels().list(part=part, id=channelID, maxResults=50)
    return request.execute()

### Get videos statistics

In [3]:
def get_videos_data(videoID):
    # Refer to the documentation: https://googleapis.github.io/google-api-python-client/docs/dyn/youtube_v3.search.html
    videos = []
    
    request = youtube.videos().list(
        part="contentDetails", id=videoID, maxResults=50)

    response = request.execute()
    responseItems = response['items']
    videos.extend(responseItems)

    # if there is nextPageToken, then keep calling the API
    while response.get('nextPageToken', None):
        request = youtube.videos().list(
        part="contentDetails", id=videoID, 
            pageToken=response['nextPageToken'],
            maxResults=50)
        
        response = request.execute()
        responseItems = response['items']
        videos.extend(responseItems)
    
    print(f"Finished fetching videos for {channelId}. {len(videos)} videos found.")

    return videos

### Get all videos from channel

In [4]:
def get_channel_videos(channelId):
    # Refer to the documentation: https://googleapis.github.io/google-api-python-client/docs/dyn/youtube_v3.search.html
    videos = []
    upload_id = "UU" + channelId[2:]
    
    request = youtube.playlistItems().list(
        part="snippet", playlistId=upload_id, maxResults=50)

    response = request.execute()
    responseItems = response['items']
    videos.extend(responseItems)

    # if there is nextPageToken, then keep calling the API
    while response.get('nextPageToken', None):
        request = youtube.playlistItems().list(
            part="snippet", 
            playlistId=upload_id, 
            pageToken=response['nextPageToken'],
            maxResults=50)
        
        response = request.execute()
        responseItems = response['items']
        videos.extend(responseItems)
    
    print(f"Finished fetching videos for {channelId}. {len(videos)} videos found.")

    return videos

### Comment threads

In [5]:
def comment_threads(channelID, to_csv=False):
    comments_list = []
    
    request = youtube.commentThreads().list(part='id,replies,snippet', videoId=channelID)
    response = request.execute()
    comments_list.extend(process_comments(response['items']))

    # if there is nextPageToken, then keep calling the API
    while response.get('nextPageToken', None):
        request = youtube.commentThreads().list(
            part='id,replies,snippet',
            videoId=channelID,
            pageToken=response['nextPageToken']
        )
        response = request.execute()
        comments_list.extend(process_comments(response['items']))
    comments_list = list(unique_everseen(comments_list))

    print(f"Finished fetching comments for {channelID}. {len(comments_list)} comments found.")
    
    if to_csv:
        make_csv(comments_list, channelID)
    
    return comments_list

## Funções estratégicas

In [7]:
# api_key = open("api_credentials").read()
api_key = open("api_credentials2").read()

In [8]:
youtube = build("youtube", "v3", developerKey=api_key)

pyscriptVidId = 'Qo8dXyKXyME'
channelId = 'UCxXL5491Db9U8Rhfs-2LVFg'

stats = channel_stats(channelId, "statistics")
branding = channel_stats(channelId, "brandingSettings")

NameError: name 'build' is not defined

### Análise de concorrentes por keyword

In [12]:
def players_by_keyword(queries, location=""):
    resp = []
    dict_players = []
    
    for query in queries:
        response = search_result(query)
        concorrentes = {i["snippet"]["channelId"]: 
                        {"canal": i["snippet"]["channelTitle"]} for i in response["items"]}

        stats = channel_stats(list(concorrentes.keys()), "statistics")
        branding = channel_stats(list(concorrentes.keys()), "brandingSettings")

        for stat in stats["items"]:
            id_ = stat["id"]
            concorrentes[id_]["views"] = int(stat["statistics"]["viewCount"])
            concorrentes[id_]["subscribers"] = int(stat["statistics"]["subscriberCount"])
            concorrentes[id_]["videos"] = int(stat["statistics"]["videoCount"])

        for brand in branding["items"]:
            id_ = brand["id"]
            brand_ = brand["brandingSettings"]["channel"]
            concorrentes[id_]["description"] = brand_["description"] if "description" in brand_.keys() else ""
            concorrentes[id_]["country"] = brand_["country"] if "country" in brand_.keys() else ""

        # Reorganiza em lista de dicionários
        
        for conc in concorrentes.keys():
            dict_conc = {"id": conc} | concorrentes[conc]
            dict_players += [dict_conc]
    
    df_players = pd.DataFrame(dict_players)
    df_players.set_index("canal", inplace=True)
    df_players.sort_values(by="subscribers", ascending=False, inplace=True) 
    
    df_players["mean_views"] = df_players["views"] / df_players["videos"] / 1000
    df_players["subscribers"] = df_players["subscribers"] / 1000
    df_players_loc = df_players[df_players["country"] == location] if location != "" else df_players
    return df_players_loc.drop_duplicates()

In [13]:
keywords = ["python"]
df_players = players_by_keyword(keywords, location="BR")

NameError: name 'youtube' is not defined

In [ ]:
df_players

In [ ]:
keywords = ["python"]
df_players = players_by_keyword(keywords, location="BR")

In [ ]:
df_players

### Análise de players por palavra chave pesquisada

In [ ]:
response = search_result("python")

In [ ]:
df_resp = pd.DataFrame(response["items"])
df_resp["channel"] = df_resp["snippet"].apply(lambda x: x["channelTitle"])
df_resp["title"] = df_resp["snippet"].apply(lambda x: x["title"])

In [ ]:
df_resp[["channel", "title"]]["channel"].value_counts()

### Dissecar vídeos de um canal

In [ ]:
channel_id = df_players.iloc[5].id

In [ ]:
channel_videos = get_channel_videos(channel_id)
df_channel_videos = pd.DataFrame(channel_videos)

In [ ]:
df_channel_videos["snippet"].iloc[0]["title"]

In [ ]:
df_channel_videos["title"] = df_channel_videos["snippet"].apply(lambda x: x["title"])
# df_channel_videos["video_id"]

In [ ]:
channel_videos

In [ ]:
df_channel_videos

In [ ]:
videos = get_videos_data(